# リバーシを作る

## 仕様
- 必須機能
  - 以下に示すような画面例を持つリバーシをプレイできる。（リバーシのルールは検索して調べること）
  - 人vs人対戦
  - 入力はキーボードからの数字とアルファべット→input()?
  - ゲーム終了時に勝敗，白と黒の石の数を表示
- 任意機能
  - 人vs ＣＰＵモード、CPU vs CPUモードの追加
  - 対戦結果の外部ファイルへの出力
  - ゲーム途中のセーブ機能とロード機能
  - その他の機能を自由に追加して構わない

## メモ
枠をつくる(8*8)
中央2*2[4,4], [4,5], [5,4], [5,5]の
○●
●○←が必要

白と黒の石（○と●）
挟んだら色が反転する
相手の色を挟める場所しか置けない
挟めなかったらパス
全部埋まるor どっちも置けない→ゲーム終了

### 作る関数
1. 盤面を表示
2. 石を置く
3. 石が置けるかをチェック

1. board関数
   tablesize*tablesizeの配列をつくる（intにする）
   初期条件：[3][3],[4][4]に白，[3][4]，[4][3]に黒
3. display関数
   配列の各要素に白，黒，空白を割り当て8*8できれいに表示
4. check関数
   盤内においたか
   →周りの8マスに相手の石があるか
    ない→faulse
    ある→その直線上の石を調べる
        自分の石→faulse
        相手の石→loop
    
            
    

In [25]:
import numpy as np

white = 1
black = -1
blank = 0
tablesize = 8

def board(tablesize):
    aaa = np.zeros((tablesize, tablesize))
    board = aaa.astype(int)
    board[3][3] = 1
    board[3][4] = -1
    board[4][3] = -1
    board[4][4] = 1
    return board

def display(board):
    for y in range(tablesize):
        for x in range(tablesize):
            if board[x][y] == 0:
                print("-", end="  ")
            elif board[x][y] == 1:
                print("○", end="  ")
            else:
                print("●", end="  ")
        print()
        
    

b = board(8)
b[3][3] = 1
print(b)
print("-------------------------------------------")
display(b)
print("----------------------------------------")
b[6][1] = -1
display(b)

[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  1 -1  0  0  0]
 [ 0  0  0 -1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
-------------------------------------------
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  ○  ●  -  -  -  
-  -  -  ●  ○  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
----------------------------------------
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  ●  -  
-  -  -  -  -  -  -  -  
-  -  -  ○  ●  -  -  -  
-  -  -  ●  ○  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
